### Spot best predictors

In [ ]:
for target_class in range(8):
    print "Class ", ssConvertMap[target_class]

    max_target = np.argmax( predictions[:, :, target_class] )
    max_target = np.unravel_index( max_target, predictions.shape[0:2] )
    seq = max_target[0]
    pos = max_target[1]
    
    print "max_target", max_target
    print "value: ", predictions[seq, pos, target_class]

    if pos >= window:
        init = pos - window
    else:
        init = 0
    end = pos + window + 1
    
    gato = decode( X_am[seq] )
    print "Aminoacids ", gato[ init:pos ], gato[ pos ], gato[ pos+1:end ]
    perro = convertPredictQ8Result2HumanReadable( predictions[seq] )
    print "Predictions", perro[init:pos], perro[pos], perro[pos+1:end]
    conejo = "".join([ssConvertMap[el] for el in labels[seq]])
    print "True labels", conejo[init:pos], conejo[pos], conejo[pos+1:end], "\n"

------------------------------------------------

## Analyze sequence predictions

In [ ]:
def calculate_seq_accuracy(predictions, labels):
    num_seq = len(mask)
    seq_len = predictions.shape[1]

    tot_acc = 0
    seq_acc = np.zeros(num_seq)
    for seq in range(num_seq):
        for pos in range(seq_len):
            if mask[seq,pos]:
                if labels[seq, pos] == np.argmax(predictions[seq, pos]):
                    seq_acc[seq] += 1
                    tot_acc +=1
            else:
                break

        seq_acc[seq] /= np.sum(mask[seq])
    
    print tot_acc / np.sum(mask)
    return seq_acc

seq_acc = calculate_seq_accuracy(predictions, labels)
seq_acc_train = seq_acc[:split_value]
seq_acc_test = seq_acc[split_value:]

lengths_train = np.sum(mask[:split_value], axis=1)
lengths_test = np.sum(mask[split_value:], axis=1)
print len(lengths_train[lengths_train > 300])
print len(lengths_test[lengths_test > 300])

### Per-sequence accuracy plots

In [1]:
def print_sorted_seqs(seqs, ax, label):
    ax.plot(sorted(seqs), marker="X", linewidth=0)
    ax.set(title='Per-sequence accuracy ('+label+')', ylim=[0, 1])

fig,ax = plt.subplots(2,2, figsize=(15,7))

print_sorted_seqs(seq_acc_train, ax[0][0], "train")
print_sorted_seqs(seq_acc_test, ax[0][1], "test")

print_sorted_seqs(seq_acc_train[lengths_train > 300], ax[1][0], 'train > 300')
print_sorted_seqs(seq_acc_test[lengths_test > 300], ax[1][1], 'test > 300')

NameError: name 'plt' is not defined

### Accuracy-size plots

In [ ]:
def print_length_vs_acc(lengths, seqs, ax, label):
    seq_len = 700
    
    ax.plot(lengths, seqs, marker="X", linewidth=0, label="sequences")
    ax.plot(np.mean(seqs)*np.ones(seq_len), label="mean")
    ax.set(title='Per-sequence accuracy ('+label+'), mean: {:.2f}'.format(np.mean(seqs)),
           ylabel="accuracy", xlabel="sequence length", ylim=[0, 1])
    ax.legend()

fig,ax = plt.subplots(2,2, figsize=(20,10))

print_length_vs_acc(lengths_train, seq_acc_train, ax[0][0], 'train')
print_length_vs_acc(lengths_test, seq_acc_test, ax[0][1], 'test')

print_length_vs_acc(lengths_train[lengths_train > 300], seq_acc_train[lengths_train > 300], ax[1][0], 'train')
print_length_vs_acc(lengths_test[lengths_test > 300], seq_acc_test[lengths_test > 300], ax[1][1], 'test')